In [2]:
!pip install rdkit

In [4]:
import os
import math
import json
import time
import requests
from collections import Counter, defaultdict
from typing import List, Tuple
import numpy as np
import pandas as pd
from tqdm import tqdm

# RDKit
from rdkit import Chem
from rdkit.Chem import AllChem

# ML
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, top_k_accuracy_score, classification_report

In [5]:
CHEMBL_ACTIVITY_API = "https://www.ebi.ac.uk/chembl/api/data/activity.json"
CHEMBL_TARGET_API = "https://www.ebi.ac.uk/chembl/api/data/target/"

PAGE_SIZE = 100  # number of records per API page (100 is reasonable)
MAX_COMPOUNDS = 5000  # keep this modest for hackathon; increase if you have time
TOP_N_TARGETS = 50  # keep to top N targets by counts (multi-class)
FINGERPRINT_RADIUS = 2
FINGERPRINT_NBITS = 2048
RANDOM_SEED = 42
MODEL_SAVE_PATH = "target_model.h5"

np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

In [6]:
def fetch_chembl_activities(limit:int=PAGE_SIZE, offset:int=0, filters:dict=None) -> dict:
    params = {"limit": limit, "offset": offset}
    if filters:
        params.update(filters)
    resp = requests.get(CHEMBL_ACTIVITY_API, params=params, timeout=30)
    resp.raise_for_status()
    return resp.json()

In [7]:
# Data assembly
def build_activity_dataframe(max_records:int=MAX_COMPOUNDS, page_size:int=PAGE_SIZE) -> pd.DataFrame:
    """
    Fetch activity records until we gather `max_records` unique molecules with pchembl_value or valid IC50.
    We'll prefer pchembl_value if present (already -log10(M)), else try to compute pIC50 from standard_value units (nM -> M).
    We construct rows: molecule_chembl_id, molecule_smiles, target_chembl_id, target_pref_name, pchembl_value
    """
    rows = []
    offset = 0
    seen_mols = set()
    pbar = tqdm(total=max_records, desc="Fetching ChEMBL activities")
    while len(rows) < max_records:
        data = fetch_chembl_activities(limit=page_size, offset=offset,
                                      filters={"pchembl_value__isnull":"false"})  # prefer pchembl precomputed
        activities = data.get("activities", []) or data.get("data", []) or []
        if not activities:
            break
        for act in activities:
            # ChEMBL shapes vary; guard for missing fields
            mol_id = act.get("molecule_chembl_id") or act.get("molecule_chembl_id")
            smiles = act.get("canonical_smiles") or act.get("molecule_structures", {}).get("canonical_smiles")
            target_id = act.get("target_chembl_id")
            pchembl = act.get("pchembl_value")
            if not (mol_id and smiles and target_id and pchembl):
                continue
            # Keep one record per molecule-target pair; allow multiple molecules
            key = (mol_id, target_id)
            # we allow multiple different molecules; but limit total rows
            rows.append({
                "molecule_chembl_id": mol_id,
                "smiles": smiles,
                "target_chembl_id": target_id,
                "pchembl_value": float(pchembl)
            })
            if len(rows) >= max_records:
                break
        offset += page_size
        pbar.update(len(rows) - pbar.n)
        # If API returns fewer than page size, likely no more data
        if len(activities) < page_size:
            break
    pbar.close()
    df = pd.DataFrame(rows)
    return df

In [8]:
# Map target_chembl_id to general target name
def fetch_target_pref_names(target_ids: List[str]) -> dict:
    """
    Use ChEMBL target endpoint to fetch `pref_name` for each target_chembl_id.
    Query each target individually (simple approach).
    """
    mapping = {}
    for t in tqdm(target_ids, desc="Fetching target names"):
        try:
            resp = requests.get(f"{CHEMBL_TARGET_API}{t}.json", timeout=20)
            if resp.status_code == 200:
                j = resp.json()
                pref = j.get("pref_name") or j.get("target_components", [{}])[0].get("component_synonyms", [{}])[0].get("component_name")
                mapping[t] = pref if pref else t
            else:
                mapping[t] = t
        except Exception:
            mapping[t] = t
    return mapping

In [9]:
# converting SMILES string into a numerical fingerprint
def smiles_to_fingerprint(smiles: str, nbits:int=FINGERPRINT_NBITS, radius:int=FINGERPRINT_RADIUS) -> np.ndarray:
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=nbits)
    arr = np.zeros((int(nbits),), dtype=np.uint8)
    AllChem.DataStructs.ConvertToNumpyArray(fp, arr)
    return arr

In [10]:
# Prepare ML dataset (choose single "most potent" target per molecule)
def assemble_dataset(df: pd.DataFrame, top_n_targets:int=TOP_N_TARGETS) -> Tuple[np.ndarray, np.ndarray, dict]:
    """
    - For each molecule, keep the target with the highest pchembl_value (most potent).
    - Then restrict to the top_n_targets by frequency.
    - Compute fingerprints and return X, y (labels), and label encoder mapping
    """
    # choose most potent target per molecule
    df_sorted = df.sort_values(["molecule_chembl_id", "pchembl_value"], ascending=[True, False])
    df_best = df_sorted.groupby("molecule_chembl_id").first().reset_index()

    # count targets, pick top N
    tcounts = df_best["target_chembl_id"].value_counts()
    top_targets = tcounts.nlargest(top_n_targets).index.tolist()
    df_top = df_best[df_best["target_chembl_id"].isin(top_targets)].copy()

    # fetch target names
    target_map = fetch_target_pref_names(sorted(df_top["target_chembl_id"].unique().tolist()))
    df_top["target_name"] = df_top["target_chembl_id"].map(target_map)

    # fingerprints
    fps = []
    bad_idx = []
    for idx, row in df_top.iterrows():
        arr = smiles_to_fingerprint(row["smiles"])
        if arr is None:
            bad_idx.append(idx)
        else:
            fps.append(arr)
    if bad_idx:
        df_top = df_top.drop(index=bad_idx).reset_index(drop=True)
    X = np.asarray(fps, dtype=np.float32)
    y_names = df_top["target_name"].values
    le = LabelEncoder()
    y = le.fit_transform(y_names)
    label_map = {i: name for i, name in enumerate(le.classes_)}

    return X, y, label_map

In [11]:
# Model
def build_model(input_dim:int, n_classes:int) -> tf.keras.Model:
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(input_dim,)),
        tf.keras.layers.Dense(1024, activation="relu"),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(512, activation="relu"),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(n_classes, activation="softmax")
    ])
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy", tf.keras.metrics.SparseTopKCategoricalAccuracy(k=3, name="top_3_acc")]
    )
    return model


In [42]:
def train_and_evaluate(X: np.ndarray, y: np.ndarray, label_map: dict):
    # train/test split stratified
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.30, random_state=RANDOM_SEED, stratify=y
    )
    print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}; #classes={len(label_map)}")

    model = build_model(input_dim=X.shape[1], n_classes=len(label_map))

    # callbacks
    es = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
    history = model.fit(
        X_train, y_train,
        validation_split=0.1,
        epochs=50,
        batch_size=64,
        callbacks=[es],
        verbose=2
    )

    # predict & evaluate
    preds_proba = model.predict(X_test)
    preds = np.argmax(preds_proba, axis=1)
    acc = accuracy_score(y_test, preds)
    try:
        top3 = top_k_accuracy_score(y_test, preds_proba, k=3, labels=list(range(len(label_map))))
    except Exception:
        top3 = None

    print(f"Test accuracy: {acc:.4f}")
    if top3 is not None:
        print(f"Top-3 accuracy: {top3:.4f}")

    print("\nClassification report (coarse):")
    print(classification_report(y_test, preds, zero_division=0))

    # save model
    model.save(MODEL_SAVE_PATH)
    print(f"Saved model to {MODEL_SAVE_PATH}")

    return model, history

In [17]:
import time
import pickle
import requests
from requests.adapters import HTTPAdapter, Retry

# Create a safe requests session with retry + longer timeout
session = requests.Session()
retries = Retry(
    total=5,                # retry up to 5 times
    backoff_factor=2,       # wait 2, 4, 8... seconds between retries
    status_forcelist=[429, 500, 502, 503, 504],
)
session.mount("https://", HTTPAdapter(max_retries=retries))

# Example: wrap your ChEMBL call with safe fetch
def safe_get(url, params=None, timeout=60):
    for _ in range(3):  # Try 3 times
        try:
            return session.get(url, params=params, timeout=timeout)
        except requests.exceptions.ReadTimeout:
            print("Timeout — retrying...")
            time.sleep(3)
        except Exception as e:
            print(f"Network error: {e}. Retrying...")
            time.sleep(3)
    print("Failed after retries — skipping this batch.")
    return None


def main():
    print("Building dataset from ChEMBL...")

    # Replace inside your build_activity_dataframe() with safe_get instead of requests.get
    df = build_activity_dataframe(max_records=MAX_COMPOUNDS, page_size=PAGE_SIZE)
    if df.empty:
        raise RuntimeError("No data fetched. Check ChEMBL API access or increase MAX_COMPOUNDS/PAGE_SIZE.")

    print(f"Fetched {len(df)} activity records. Unique molecules: {df['molecule_chembl_id'].nunique()}")
    X, y, label_map = assemble_dataset(df, top_n_targets=TOP_N_TARGETS)

    with open('label_map.pkl', 'wb') as f:
        pickle.dump(label_map, f)

    print(f"Prepared dataset: X.shape={X.shape}, #classes={len(label_map)}")
    if X.shape[0] < 100 or len(label_map) < 2:
        print("Warning: dataset is small — consider increasing MAX_COMPOUNDS or TOP_N_TARGETS for better training.")

#    model, history = train_and_evaluate(X, y, label_map)
    print("Done.")

if __name__ == "__main__":
    main()


Building dataset from ChEMBL...


Fetching ChEMBL activities: 100%|██████████| 5000/5000 [03:18<00:00, 25.23it/s]


Fetched 5000 activity records. Unique molecules: 2543


Fetching target names: 100%|██████████| 50/50 [01:29<00:00,  1.79s/it]
[00:53:08] DEPRECATION WARNING: please use MorganGenerator
[00:53:08] DEPRECATION WARNING: please use MorganGenerator
[00:53:08] DEPRECATION WARNING: please use MorganGenerator
[00:53:08] DEPRECATION WARNING: please use MorganGenerator
[00:53:08] DEPRECATION WARNING: please use MorganGenerator
[00:53:08] DEPRECATION WARNING: please use MorganGenerator
[00:53:08] DEPRECATION WARNING: please use MorganGenerator
[00:53:08] DEPRECATION WARNING: please use MorganGenerator
[00:53:08] DEPRECATION WARNING: please use MorganGenerator
[00:53:08] DEPRECATION WARNING: please use MorganGenerator
[00:53:08] DEPRECATION WARNING: please use MorganGenerator
[00:53:08] DEPRECATION WARNING: please use MorganGenerator
[00:53:08] DEPRECATION WARNING: please use MorganGenerator
[00:53:08] DEPRECATION WARNING: please use MorganGenerator
[00:53:08] DEPRECATION WARNING: please use MorganGenerator
[00:53:08] DEPRECATION WARNING: please use M

Prepared dataset: X.shape=(1332, 2048), #classes=49
Done.
